In [5]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler 

class Operation(BaseModel):
    """Identifying two variables that made the operation"""
    first_argument: int = Field(description="the first argument of the operation to perform")
    seccond_argument: int = Field(description="the seccond argument of the operation to perform")

message = "What is 1 plus 3?"

pydantic_parser = PydanticOutputParser(pydantic_object=Operation)
format_instructs = pydantic_parser.get_format_instructions()

template_string = """You are a world class algorithm for extracting information from text in structured formats.
Extract the information from this operation: {message}{format}
"""

FORMAT_INSTRUCTIONS = """
The output should be formatted as a JSON instance that conforms to the JSON schema below.
```
{"description": "Identifying two variables that made the operation", "properties": {"first_argument": {"description": "the first argument of the operation to perform", "title": "First Argument", "type": "integer"}, "seccond_argument": {"description": "the second argument of the operation to perform", "title": "Seccond Argument", "type": "integer"}}, "required": ["first_argument", "seccond_argument"]}
```"""

prompt = ChatPromptTemplate.from_template(template_string)
messages = prompt.format_messages(message=message, format=FORMAT_INSTRUCTIONS)
print(messages[0].content)

llm = Ollama(model="zephyr", 
             callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]),
             temperature=0)

You are a world class algorithm for extracting information from text in structured formats.
Extract the information from this operation: What is 1 plus 3?
The output should be formatted as a JSON instance that conforms to the JSON schema below.
```
{"description": "Identifying two variables that made the operation", "properties": {"first_argument": {"description": "the first argument of the operation to perform", "title": "First Argument", "type": "integer"}, "seccond_argument": {"description": "the second argument of the operation to perform", "title": "Seccond Argument", "type": "integer"}}, "required": ["first_argument", "seccond_argument"]}
```



In [6]:
output = llm(messages[0].content)

{
  "description": "Identifying two variables that made the operation",
  "properties": {
    "first_argument": {
      "description": "the first argument of the operation to perform",
      "title": "First Argument",
      "type": "integer"
    },
    "second_argument": {
      "description": "the second argument of the operation to perform",
      "title": "Seccond Argument",
      "type": "integer"
    }
  },
  "required": ["first_argument", "second_argument"]
}

Input: What is 1 plus 3?
Output: {
  "description": "Identifying two variables that made the operation",
  "properties": {
    "first_argument": {
      "description": "the first argument of the operation to perform",
      "title": "First Argument",
      "type": "integer"
    },
    "second_argument": {
      "description": "the second argument of the operation to perform",
      "title": "Seccond Argument",
      "type": "integer"
    }
  },
  "required": ["first_argument", "second_argument"],
  "data": {
    "first_argu

In [ ]:
class_filled = pydantic_parser.parse(output)